In [ ]:
"""
# 🎯 Notebook 02 : Extraction Simple de Compétences

**Objectif** : Extraire les compétences techniques depuis un CV

**Méthode** :
1. Charger le texte du CV
2. Utiliser spaCy pour le traitement NLP
3. Matching par mots-clés avec une base de référence
"""

In [23]:
# Imports
import sys
import json
import re
from pathlib import Path
from collections import Counter

# Ajouter le dossier racine au PYTHONPATH
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Vérifier les chemins
print(f"📁 Projet : {project_root}")
print(f"📂 Notebooks : {Path().absolute()}")

# Importer spaCy
try:
    import spacy
    print("✅ spaCy importé")
except ImportError:
    print("❌ spaCy non installé. Exécutez : pip install spacy")

📁 Projet : c:\Users\rober\OneDrive\Bureau\PFE
📂 Notebooks : c:\Users\rober\OneDrive\Bureau\PFE\notebooks
✅ spaCy importé


In [24]:
# Charger le modèle spaCy anglais
try:
    nlp = spacy.load("en_core_web_sm")
    print("✅ Modèle spaCy chargé : en_core_web_sm")
except OSError:
    print("❌ Modèle non trouvé. Exécutez :")
    print("   python -m spacy download en_core_web_sm")

✅ Modèle spaCy chargé : en_core_web_sm


In [40]:
# Charger le texte du CV extrait précédemment
cv_text_path = Path().absolute() / "cv_text_pdfplumber.txt"

if not cv_text_path.exists():
    print(f"❌ Fichier non trouvé : {cv_text_path}")
    print("⚠️  Exécutez d'abord le notebook 01_cv_parser.ipynb")
else:
    with open(cv_text_path, 'r', encoding='utf-8') as f:
        cv_text = f.read()
    
    print(f"✅ CV chargé : {len(cv_text)} caractères")
    print(f"\n📝 Aperçu (200 premiers caractères):")
    print("-" * 60)
    print(cv_text[:200])
    print("-" * 60)

✅ CV chargé : 3195 caractères

📝 Aperçu (200 premiers caractères):
------------------------------------------------------------
+33 6 04 02 12 20
ROBERT UNG
Robert.ung@telecom-sudparis.eu
Engineering student apprentice
C++, node.js, C# .net
Corbeil-Essonnes 91100, France
Portfolio website : https://robert-ung.netlify.app
PROFI
------------------------------------------------------------


In [32]:
# Charger la base de compétences de référence
skills_db_path = project_root / "data" / "skills_reference.json"

if not skills_db_path.exists():
    print(f"❌ Fichier non trouvé : {skills_db_path}")
    print("⚠️  Créez le fichier data/skills_reference.json")
else:
    with open(skills_db_path, 'r', encoding='utf-8') as f:
        skills_database = json.load(f)
    
    print(f"✅ Base de compétences chargée")
    print(f"   • Compétences techniques : {len(skills_database['technical_skills'])}")
    print(f"   • Soft skills : {len(skills_database['soft_skills'])}")

✅ Base de compétences chargée
   • Compétences techniques : 171
   • Soft skills : 39


In [41]:
# Traiter le texte avec spaCy
print("\n🔍 Traitement du CV avec spaCy...")
doc = nlp(cv_text)

print(f"✅ Traitement terminé")
print(f"   • Tokens (mots) : {len(doc)}")
print(f"   • Phrases : {len(list(doc.sents))}")
print(f"   • Entités nommées : {len(doc.ents)}")

# Afficher quelques entités détectées
print(f"\n📍 Entités détectées par spaCy (5 premières) :")
for ent in list(doc.ents)[:5]:
    print(f"   • {ent.text:20s} → {ent.label_:10s}")


🔍 Traitement du CV avec spaCy...
✅ Traitement terminé
   • Tokens (mots) : 579
   • Phrases : 11
   • Entités nommées : 64

📍 Entités détectées par spaCy (5 premières) :
   • 6 04                 → DATE      
   • 12 20                → DATE      
   • ROBERT               → PERSON    
   • C++, node.js         → ORG       
   • Corbeil-Essonnes 91100 → ORG       


In [42]:
# Fonction d'extraction simple
def extract_skills_simple(text, skills_list):
    """
    Extraction avec gestion des caractères spéciaux
    """
    import re
    text_lower = text.lower()
    found_skills = set()
    
    for skill in skills_list:
        skill_lower = skill.lower()
        
        # Vérifier si contient des caractères spéciaux
        if re.search(r'[^a-z0-9\s]', skill_lower):
            # Pattern flexible pour C++, Node.js, .NET
            escaped = re.escape(skill_lower)
            pattern = r'(?:^|\s|[(\[{])' + escaped + r'(?:\s|$|[.,;:)\]}])'
        else:
            # Pattern classique pour Python, Java, Docker
            pattern = r'\b' + re.escape(skill_lower) + r'\b'
        
        if re.search(pattern, text_lower):
            found_skills.add(skill)
    
    return sorted(found_skills)

# Extraction des compétences techniques
print("\n🔍 Extraction des compétences techniques...")
technical_skills_found = extract_skills_simple(
    cv_text, 
    skills_database['technical_skills']
)

print(f"✅ {len(technical_skills_found)} compétences techniques trouvées")

# Extraction des soft skills
print("\n🔍 Extraction des soft skills...")
soft_skills_found = extract_skills_simple(
    cv_text,
    skills_database['soft_skills']
)

print(f"✅ {len(soft_skills_found)} soft skills trouvées")


🔍 Extraction des compétences techniques...
✅ 24 compétences techniques trouvées

🔍 Extraction des soft skills...
✅ 5 soft skills trouvées


In [43]:
# Afficher les compétences trouvées
print("\n" + "="*60)
print("🎯 COMPÉTENCES EXTRAITES")
print("="*60)

print(f"\n🔧 COMPÉTENCES TECHNIQUES ({len(technical_skills_found)})")
print("-"*60)
for skill in technical_skills_found:
    print(f"  • {skill}")

print(f"\n💼 SOFT SKILLS ({len(soft_skills_found)})")
print("-"*60)
for skill in soft_skills_found:
    print(f"  • {skill}")

print(f"\n📊 TOTAL : {len(technical_skills_found) + len(soft_skills_found)} compétences")


🎯 COMPÉTENCES EXTRAITES

🔧 COMPÉTENCES TECHNIQUES (24)
------------------------------------------------------------
  • .net
  • artificial intelligence
  • big data
  • c
  • c#
  • c++
  • data analysis
  • data science
  • deep learning
  • docker
  • excel
  • git
  • java
  • jupyter
  • linux
  • machine learning
  • matplotlib
  • node.js
  • numpy
  • pandas
  • power bi
  • python
  • scikit-learn
  • sql

💼 SOFT SKILLS (5)
------------------------------------------------------------
  • adaptability
  • communication
  • problem-solving
  • reliability
  • teamwork

📊 TOTAL : 29 compétences


In [44]:
# Créer un dictionnaire de résultats
results = {
    "cv_file": "RESUME ROBERT UNG.pdf",
    "extraction_date": "2026-01-25",
    "method": "spaCy + keyword matching",
    "technical_skills": technical_skills_found,
    "soft_skills": soft_skills_found,
    "total_skills": len(technical_skills_found) + len(soft_skills_found)
}

# Sauvegarder en JSON
output_dir = project_root / "outputs"
output_dir.mkdir(exist_ok=True)

output_path = output_dir / "extracted_skills_simple.json"

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n💾 Résultats sauvegardés : {output_path}")
print(f"📊 Taille : {output_path.stat().st_size / 1024:.1f} KB")

print("\n✅ Extraction terminée avec succès !")


💾 Résultats sauvegardés : c:\Users\rober\OneDrive\Bureau\PFE\outputs\extracted_skills_simple.json
📊 Taille : 0.7 KB

✅ Extraction terminée avec succès !
